In [0]:
%sql
USE CATALOG mvp;
USE SCHEMA silver;

## Modelagem

A partir da tabela voos iremos montar nosso esquema estrela. O esquema será composto de 5 dimensões e um fato:

#### Dimensão aeronave:
- **id_equipamento (chave primária)**
- sg_equipamento_icao
- ds_modelo
- ds_matricula

#### Dimensão aeroporto:
- **id_aerodromo (chave primária)**
- id_aerodromo
- sg_icao
- sg_iata
- nm_aerodromo
- nm_municipio
- sg_uf
- nm_regiao
- nm_pais
- nm_continente

#### Dimensão empresa:
- **id_empresa (chave primária)**
- sg_empresa_icao
- sg_empresa_iata
- nm_empresa
- nm_pais
- ds_tipo_empresa

#### Dimensão linha:
- **id_tipo_linha (chave primária)**
- cd_tipo_linha
- ds_tipo_linha
- ds_natureza_tipo_linha
- ds_servico_tipo_linha

#### Dimensão tempo:
- **dt_voo (chave primária)**
- nr_dia
- nr_mes
- nr_ano
- nr_semestre
- nr_trimestre intege

## Aplicando filtros e critérios de qualidade

Para entender melhor o motivo dessas operações, consulte esta planilha [aqui](https://docs.google.com/spreadsheets/d/1_4_-XutVG2gyGrHMkfTlin1-UbQRA27tdRVC9ZI0fmk/edit?usp=sharing). 
1. Selecionar colunas necessárias (consultar motivo para colunas serem descartadas na planilha a cima);
2. Aplicar filtros e correção de valores inválidos;
3. Aplicar tipagem correta nas colunas;
4. Montar entidades dimensões;
5. Montar entidade fato;


In [0]:
import pyspark.sql.functions as F

In [0]:
df = spark.table("mvp.bronze.voos")

### Seleção das colunas necessárias:

In [0]:
df_colunas = df.select([
  "id_equipamento",
  "sg_equipamento_icao",
  "ds_modelo",
  "ds_matricula",
  "id_aerodromo_origem",
  "sg_icao_origem",
  "sg_iata_origem",
  "nm_aerodromo_origem",
  "nm_municipio_origem",
  "sg_uf_origem",
  "nm_regiao_origem",
  "nm_pais_origem",
  "nm_continente_origem",
  "id_aerodromo_destino",
  "sg_icao_destino",
  "sg_iata_destino",
  "nm_aerodromo_destino",
  "nm_municipio_destino",
  "sg_uf_destino",
  "nm_regiao_destino",
  "nm_pais_destino",
  "nm_continente_destino",
  "dt_partida_real",
  "id_empresa",
  "sg_empresa_icao",
  "sg_empresa_iata",
  "nm_empresa",
  "nm_pais",
  "id_tipo_linha",
  "cd_tipo_linha",
  "ds_tipo_linha",
  "ds_natureza_tipo_linha",
  "ds_servico_tipo_linha",
  "id_basica",
  "nr_voo",
  "nr_singular",
  "ds_natureza_etapa",
  "dt_chegada_real",
  "lt_combustivel",
  "nr_assentos_ofertados",
  "kg_payload",
  "km_distancia",
  "nr_passag_pagos",
  "nr_passag_gratis",
  "kg_bagagem_livre",
  "kg_bagagem_excesso",
  "kg_carga_paga",
  "kg_carga_gratis",
  "kg_correio",
  "nr_horas_voadas",
  "kg_peso",
  "nr_velocidade_media",
  "nr_pax_gratis_km",
  "nr_carga_paga_km",
  "nr_carga_gratis_km",
  "nr_correio_km",
  "nr_bagagem_paga_km",
  "nr_bagagem_gratis_km",
  "nr_ask",
  "nr_rpk",
  "nr_atk"
])

### Aplicação de filtros e correção de valores inválidos:

In [0]:
print("Excluindo linhas nulas:")
print("="*20)
colunas = [
  "id_equipamento",
  "id_aerodromo_origem", 
  "id_aerodromo_destino",
  "id_empresa",
  "id_tipo_linha", 
  "id_basica", 
]
df_nao_nulos = df_colunas.selectExpr("*")
for coluna in colunas:
  print("Total de linhas antes da exclusão: ", df_nao_nulos.count())
  
  total_registros_nulos = df_nao_nulos.where(F.col(coluna).isNull()).count()
  print("Total de linhas nulas na coluna ", coluna, ": ", total_registros_nulos)
  
  df_nao_nulos = df_nao_nulos.filter(F.col(coluna).isNotNull())

  print("Total de linhas depois da exclusão: ", df_nao_nulos.count())
  print("-"*20)

Excluindo linhas nulas:
Total de linhas antes da exclusão:  3786077
Total de linhas nulas na coluna  id_equipamento :  0
Total de linhas depois da exclusão:  3786077
--------------------
Total de linhas antes da exclusão:  3786077
Total de linhas nulas na coluna  id_aerodromo_origem :  0
Total de linhas depois da exclusão:  3786077
--------------------
Total de linhas antes da exclusão:  3786077
Total de linhas nulas na coluna  id_aerodromo_destino :  0
Total de linhas depois da exclusão:  3786077
--------------------
Total de linhas antes da exclusão:  3786077
Total de linhas nulas na coluna  id_empresa :  0
Total de linhas depois da exclusão:  3786077
--------------------
Total de linhas antes da exclusão:  3786077
Total de linhas nulas na coluna  id_tipo_linha :  0
Total de linhas depois da exclusão:  3786077
--------------------
Total de linhas antes da exclusão:  3786077
Total de linhas nulas na coluna  id_basica :  0
Total de linhas depois da exclusão:  3786077
------------------

In [0]:
print("Convertendo strings vazias em null:")
print("="*20)
colunas = [
  "ds_modelo",
  "ds_matricula",
  "nm_aerodromo_origem",
  "nm_municipio_origem",
  "nm_regiao_origem",
  "nm_pais_origem",
  "nm_continente_origem",
  "nm_aerodromo_destino",
  "nm_municipio_destino",
  "nm_regiao_destino",
  "nm_pais_destino",
  "nm_continente_destino",
  "nm_empresa",
  "nm_pais",
  "ds_tipo_linha",
  "ds_natureza_tipo_linha",
  "nr_singular"
]
df_sem_vazios = df_nao_nulos.selectExpr("*")
for coluna in colunas:
  print(f"Total de linhas da coluna {coluna} com strings vazias ANTES do processamento: ", df_sem_vazios.where(F.trim(F.col(coluna)) == "").count())
  df_sem_vazios = df_sem_vazios.withColumn(coluna, F.when((F.trim(F.col(coluna)) == ""), F.lit(None)).otherwise(F.trim(F.col(coluna))))
  print(f"Total de linhas da coluna {coluna} com strings vazias DEPOIS do processamento: ", df_sem_vazios.where(F.trim(F.col(coluna)) == "").count())
  print("-"*20)


Convertendo strings vazias em null:
Total de linhas da coluna ds_modelo com strings vazias ANTES do processamento:  0
Total de linhas da coluna ds_modelo com strings vazias DEPOIS do processamento:  0
--------------------
Total de linhas da coluna ds_matricula com strings vazias ANTES do processamento:  0
Total de linhas da coluna ds_matricula com strings vazias DEPOIS do processamento:  0
--------------------
Total de linhas da coluna nm_aerodromo_origem com strings vazias ANTES do processamento:  0
Total de linhas da coluna nm_aerodromo_origem com strings vazias DEPOIS do processamento:  0
--------------------
Total de linhas da coluna nm_municipio_origem com strings vazias ANTES do processamento:  0
Total de linhas da coluna nm_municipio_origem com strings vazias DEPOIS do processamento:  0
--------------------
Total de linhas da coluna nm_regiao_origem com strings vazias ANTES do processamento:  35
Total de linhas da coluna nm_regiao_origem com strings vazias DEPOIS do processament

In [0]:
print("Removendo valores inteiros menores que 0:")
print("="*20)
from pyspark.sql.types import IntegerType
colunas = [
    "nr_voo",
    "lt_combustivel",
    "nr_assentos_ofertados",
    "kg_payload",
    "km_distancia",
    "nr_passag_pagos",
    "nr_passag_gratis",
    "kg_bagagem_livre",
    "kg_bagagem_excesso",
    "kg_carga_paga",
    "kg_carga_gratis",
    "kg_correio",
    "kg_peso",
    "nr_pax_gratis_km",
    "nr_carga_paga_km",
    "nr_carga_gratis_km",
    "nr_correio_km",
    "nr_bagagem_paga_km",
    "nr_bagagem_gratis_km",
    "nr_ask",
    "nr_rpk",
    "nr_atk"
]
df_sem_inteiros_menores_zero = df_sem_vazios.selectExpr("*")
for coluna in colunas:
    df_sem_inteiros_menores_zero = df_sem_inteiros_menores_zero.withColumn(coluna, F.col(coluna).try_cast(IntegerType()))
    print(f"Total de valores menores que zero na coluna {coluna} ANTES do processamento: ", df_sem_inteiros_menores_zero.where(F.col(coluna) < 0 ).count())
    df_sem_inteiros_menores_zero = df_sem_inteiros_menores_zero.withColumn(coluna, F.when((F.col(coluna) < 0), F.lit(None)).otherwise(F.col(coluna)))
    print(f"Total de valores menores que zero na coluna {coluna} ANTES do processamento: ", df_sem_inteiros_menores_zero.where(F.col(coluna) < 0 ).count())
    print("-"*20)

Removendo valores inteiros menores que 0:
Total de valores menores que zero na coluna nr_voo ANTES do processamento:  0
Total de valores menores que zero na coluna nr_voo ANTES do processamento:  0
--------------------
Total de valores menores que zero na coluna lt_combustivel ANTES do processamento:  0
Total de valores menores que zero na coluna lt_combustivel ANTES do processamento:  0
--------------------
Total de valores menores que zero na coluna nr_assentos_ofertados ANTES do processamento:  0
Total de valores menores que zero na coluna nr_assentos_ofertados ANTES do processamento:  0
--------------------
Total de valores menores que zero na coluna kg_payload ANTES do processamento:  0
Total de valores menores que zero na coluna kg_payload ANTES do processamento:  0
--------------------
Total de valores menores que zero na coluna km_distancia ANTES do processamento:  0
Total de valores menores que zero na coluna km_distancia ANTES do processamento:  0
--------------------
Total d

In [0]:
print("Removendo valores float menores que 0:")
print("="*20)
from pyspark.sql.types import FloatType
colunas = [
    "nr_horas_voadas",
    "nr_velocidade_media"
]
df_sem_float_menores_zero = df_sem_vazios.selectExpr("*")
for coluna in colunas:
    df_sem_float_menores_zero = df_sem_float_menores_zero.withColumn(coluna, F.col(coluna).try_cast(FloatType()))
    print(f"Total de valores menores que zero na coluna {coluna} ANTES do processamento: ", df_sem_float_menores_zero.where(F.col(coluna) < 0 ).count())
    df_sem_float_menores_zero = df_sem_float_menores_zero.withColumn(coluna, F.when((F.col(coluna) < 0), F.lit(None)).otherwise(F.col(coluna)))
    print(f"Total de valores menores que zero na coluna {coluna} ANTES do processamento: ", df_sem_float_menores_zero.where(F.col(coluna) < 0 ).count())
    print("-"*20)

Removendo valores float menores que 0:
Total de valores menores que zero na coluna nr_horas_voadas ANTES do processamento:  0
Total de valores menores que zero na coluna nr_horas_voadas ANTES do processamento:  0
--------------------
Total de valores menores que zero na coluna nr_velocidade_media ANTES do processamento:  0
Total de valores menores que zero na coluna nr_velocidade_media ANTES do processamento:  0
--------------------


In [0]:
df_valores_validos = df_sem_float_menores_zero.selectExpr("*")
#Valores válidos:
coluna = "cd_tipo_linha"
print("Processamento da coluna:", coluna)
valores_validos = ["N", "C", "I", "G", "X"]
print("Total de valores inválidos ANTES do processamento:", df_valores_validos.where(~(F.trim(F.col(coluna)).isin(valores_validos))).count())

df_valores_validos = df_valores_validos.withColumn(coluna, F.when(~(F.trim(F.col(coluna)).isin(valores_validos)), F.lit("X")).otherwise(F.trim(F.col(coluna))))

print("Total de valores inválidos DEPOIS do processamento:", df_valores_validos.where(~(F.trim(F.col(coluna)).isin(valores_validos))).count())


Processamento da coluna: cd_tipo_linha
Total de valores inválidos ANTES do processamento: 2
Total de valores inválidos DEPOIS do processamento: 0


In [0]:
coluna = "ds_tipo_linha"
print("Processamento da coluna:", coluna)
valores_validos = [
    "DOMÉSTICA MISTA",
    "DOMÉSTICA CARGUEIRA",
    "INTERNACIONAL CARGUEIRA",
    "INTERNACIONAL MISTA",
    "NÃO IDENTIFICADA"
]
print("Total de valores inválidos ANTES do processamento:", df_valores_validos.where(~(F.trim(F.col(coluna)).isin(valores_validos))).count())

df_valores_validos = df_valores_validos.withColumn(coluna, F.when(~(F.trim(F.col(coluna)).isin(valores_validos)), F.lit("NÃO IDENTIFICADA")).otherwise(F.trim(F.col(coluna))))

print("Total de valores inválidos DEPOIS do processamento:", df_valores_validos.where(~(F.trim(F.col(coluna)).isin(valores_validos))).count())


Processamento da coluna: ds_tipo_linha
Total de valores inválidos ANTES do processamento: 2
Total de valores inválidos DEPOIS do processamento: 0


In [0]:
coluna = "ds_natureza_etapa"
print("Processamento da coluna:", coluna)
valores_validos = [
    "DOMÉSTICA",
    "INTERNACIONAL",
    "NÃO IDENTIFICADA"
]
print("Total de valores inválidos ANTES do processamento:", df_valores_validos.where(~(F.trim(F.col(coluna)).isin(valores_validos))).count())

df_valores_validos = df_valores_validos.withColumn(coluna, F.when(~(F.trim(F.col(coluna)).isin(valores_validos)), F.lit("NÃO IDENTIFICADA")).otherwise(F.trim(F.col(coluna))))

print("Total de valores inválidos DEPOIS do processamento:", df_valores_validos.where(~(F.trim(F.col(coluna)).isin(valores_validos))).count())


Processamento da coluna: ds_natureza_etapa
Total de valores inválidos ANTES do processamento: 2
Total de valores inválidos DEPOIS do processamento: 0


In [0]:
coluna = "ds_servico_tipo_linha"
print("Processamento da coluna:", coluna)
valores_validos = [
    "NÃO IDENTIFICADO",
    "PASSAGEIRO",
    "CARGUEIRO"
]
print("Total de valores inválidos ANTES do processamento:", df_valores_validos.where(~(F.trim(F.col(coluna)).isin(valores_validos))).count())

df_valores_validos = df_valores_validos.withColumn(coluna, F.when(~(F.trim(F.col(coluna)).isin(valores_validos)), F.lit("NÃO IDENTIFICADO")).otherwise(F.trim(F.col(coluna))))

print("Total de valores inválidos DEPOIS do processamento:", df_valores_validos.where(~(F.trim(F.col(coluna)).isin(valores_validos))).count())


Processamento da coluna: ds_servico_tipo_linha
Total de valores inválidos ANTES do processamento: 2
Total de valores inválidos DEPOIS do processamento: 0


In [0]:
coluna = "dt_partida_real"
print("Processamento da coluna:", coluna)

df_valores_validos = df_valores_validos.withColumn(coluna, F.try_to_date(F.col(coluna), "yyyy-MM-dd"))

print("Total de valores inválidos ANTES do processamento:", df_valores_validos.where(F.col(coluna).isNull()).count())

df_valores_validos = df_valores_validos.filter(F.col(coluna).isNotNull())

print("Total de valores inválidos ANTES do processamento:", df_valores_validos.where(F.col(coluna).isNull()).count())



Processamento da coluna: dt_partida_real
Total de valores inválidos ANTES do processamento: 2
Total de valores inválidos ANTES do processamento: 0


In [0]:
coluna = "dt_chegada_real"
print("Processamento da coluna:", coluna)

df_valores_validos = df_valores_validos.withColumn(coluna, F.try_to_date(F.col(coluna), "yyyy-MM-dd"))

print("Total de valores inválidos ANTES do processamento:", df_valores_validos.where(F.col(coluna).isNull()).count())

df_valores_validos = df_valores_validos.filter(F.col(coluna).isNotNull())

print("Total de valores inválidos ANTES do processamento:", df_valores_validos.where(F.col(coluna).isNull()).count())



Processamento da coluna: dt_chegada_real
Total de valores inválidos ANTES do processamento: 0
Total de valores inválidos ANTES do processamento: 0


In [0]:
from pyspark.sql.types import StringType
df_valores_validos_len = df_valores_validos.selectExpr("*")
colunas_len = {
    "sg_uf_origem": 2,
    "sg_uf_destino": 2,
    "sg_empresa_iata": 2,
    "sg_iata_origem": 3,
    "sg_iata_destino": 3,
    "sg_empresa_icao": 3,
    "sg_icao_origem": 4,
    "sg_icao_destino": 4
}
for coluna, tamanho in colunas_len.items():
    print("Processamento da coluna:", coluna)
    print("Tamanho válido:", tamanho)

    print("Total de valores inválidos ANTES do processamento:", df_valores_validos_len.where(~(F.length(F.trim(F.col(coluna))) == tamanho)).count())

    df_valores_validos_len = df_valores_validos_len.withColumn(coluna, F.when(~(F.length(F.trim(F.col(coluna))) == tamanho), F.lit(None)).otherwise(F.trim(F.col(coluna))))

    print("Total de valores inválidos DEPOIS do processamento:", df_valores_validos_len.where(~(F.length(F.trim(F.col(coluna))) == tamanho)).count())

Processamento da coluna: sg_uf_origem
Tamanho válido: 2
Total de valores inválidos ANTES do processamento: 35
Total de valores inválidos DEPOIS do processamento: 0
Processamento da coluna: sg_uf_destino
Tamanho válido: 2
Total de valores inválidos ANTES do processamento: 31
Total de valores inválidos DEPOIS do processamento: 0
Processamento da coluna: sg_empresa_iata
Tamanho válido: 2
Total de valores inválidos ANTES do processamento: 16
Total de valores inválidos DEPOIS do processamento: 0
Processamento da coluna: sg_iata_origem
Tamanho válido: 3
Total de valores inválidos ANTES do processamento: 0
Total de valores inválidos DEPOIS do processamento: 0
Processamento da coluna: sg_iata_destino
Tamanho válido: 3
Total de valores inválidos ANTES do processamento: 0
Total de valores inválidos DEPOIS do processamento: 0
Processamento da coluna: sg_empresa_icao
Tamanho válido: 3
Total de valores inválidos ANTES do processamento: 0
Total de valores inválidos DEPOIS do processamento: 0
Process

In [0]:
coluna = "sg_equipamento_icao"
print("Processamento da coluna:", coluna)
min = 2
max = 4


print("Total de valores inválidos ANTES do processamento:", df_valores_validos_len.where(~((F.length(F.trim(F.col(coluna))) >= min) & (F.length(F.trim(F.col(coluna))) <= max))).count())

df_valores_validos_len = df_valores_validos_len.withColumn(coluna, F.when(~((F.length(F.trim(F.col(coluna))) >= min) & (F.length(F.trim(F.col(coluna))) <= max)), F.lit(None)).otherwise(F.trim(F.col(coluna))))

print("Total de valores inválidos DEPOIS do processamento:", df_valores_validos_len.where(~((F.length(F.trim(F.col(coluna))) >= min) & (F.length(F.trim(F.col(coluna))) <= max))).count())

Processamento da coluna: sg_equipamento_icao
Total de valores inválidos ANTES do processamento: 0
Total de valores inválidos DEPOIS do processamento: 0


### Aplicação da tipagem nas colunas:

In [0]:
from pyspark.sql.types import StringType, DateType, FloatType, IntegerType 
df = df_valores_validos_len.selectExpr("*")
df_tipado = df.select(
    F.col("cd_tipo_linha").cast(StringType()),
    F.col("ds_matricula").cast(StringType()),
    F.col("ds_modelo").cast(StringType()),
    F.col("ds_natureza_etapa").cast(StringType()),
    F.col("ds_natureza_tipo_linha").cast(StringType()),
    F.col("ds_servico_tipo_linha").cast(StringType()),
    F.col("ds_tipo_linha").cast(StringType()),
    F.col("nm_aerodromo_destino").cast(StringType()),
    F.col("nm_aerodromo_origem").cast(StringType()),
    F.col("nm_continente_destino").cast(StringType()),
    F.col("nm_continente_origem").cast(StringType()),
    F.col("nm_empresa").cast(StringType()),
    F.col("nm_municipio_destino").cast(StringType()),
    F.col("nm_municipio_origem").cast(StringType()),
    F.col("nm_pais").cast(StringType()),
    F.col("nm_pais_destino").cast(StringType()),
    F.col("nm_pais_origem").cast(StringType()),
    F.col("nm_regiao_destino").cast(StringType()),
    F.col("nm_regiao_origem").cast(StringType()),
    F.col("nr_singular").cast(StringType()),
    F.col("sg_empresa_iata").cast(StringType()),
    F.col("sg_empresa_icao").cast(StringType()),
    F.col("sg_equipamento_icao").cast(StringType()),
    F.col("sg_iata_destino").cast(StringType()),
    F.col("sg_iata_origem").cast(StringType()),
    F.col("sg_icao_destino").cast(StringType()),
    F.col("sg_icao_origem").cast(StringType()),
    F.col("sg_uf_destino").cast(StringType()),
    F.col("sg_uf_origem").cast(StringType()),

    F.col("id_aerodromo_destino").cast(IntegerType()),
    F.col("id_aerodromo_origem").cast(IntegerType()),
    F.col("id_basica").cast(IntegerType()),
    F.col("id_empresa").cast(IntegerType()),
    F.col("id_equipamento").cast(IntegerType()),
    F.col("id_tipo_linha").cast(IntegerType()),
    F.col("kg_bagagem_excesso").cast(IntegerType()),
    F.col("kg_bagagem_livre").cast(IntegerType()),
    F.col("kg_carga_gratis").cast(IntegerType()),
    F.col("kg_carga_paga").cast(IntegerType()),
    F.col("kg_correio").cast(IntegerType()),
    F.col("kg_payload").cast(IntegerType()),
    F.col("kg_peso").cast(IntegerType()),
    F.col("km_distancia").cast(IntegerType()),
    F.col("lt_combustivel").cast(IntegerType()),
    F.col("nr_ask").cast(IntegerType()),
    F.col("nr_assentos_ofertados").cast(IntegerType()),
    F.col("nr_atk").cast(IntegerType()),
    F.col("nr_bagagem_gratis_km").cast(IntegerType()),
    F.col("nr_bagagem_paga_km").cast(IntegerType()),
    F.col("nr_carga_gratis_km").cast(IntegerType()),
    F.col("nr_carga_paga_km").cast(IntegerType()),
    F.col("nr_correio_km").cast(IntegerType()),
    F.col("nr_passag_gratis").cast(IntegerType()),
    F.col("nr_passag_pagos").cast(IntegerType()),
    F.col("nr_pax_gratis_km").cast(IntegerType()),
    F.col("nr_rpk").cast(IntegerType()),
    F.col("nr_voo").cast(IntegerType()),

    F.col("nr_horas_voadas").cast(FloatType()),
    F.col("nr_velocidade_media").cast(FloatType()),

    F.col("dt_chegada_real").cast(DateType()),
    F.col("dt_partida_real").cast(DateType())
)

In [0]:
df_tipado.printSchema()

root
 |-- cd_tipo_linha: string (nullable = true)
 |-- ds_matricula: string (nullable = true)
 |-- ds_modelo: string (nullable = true)
 |-- ds_natureza_etapa: string (nullable = true)
 |-- ds_natureza_tipo_linha: string (nullable = true)
 |-- ds_servico_tipo_linha: string (nullable = true)
 |-- ds_tipo_linha: string (nullable = true)
 |-- nm_aerodromo_destino: string (nullable = true)
 |-- nm_aerodromo_origem: string (nullable = true)
 |-- nm_continente_destino: string (nullable = true)
 |-- nm_continente_origem: string (nullable = true)
 |-- nm_empresa: string (nullable = true)
 |-- nm_municipio_destino: string (nullable = true)
 |-- nm_municipio_origem: string (nullable = true)
 |-- nm_pais: string (nullable = true)
 |-- nm_pais_destino: string (nullable = true)
 |-- nm_pais_origem: string (nullable = true)
 |-- nm_regiao_destino: string (nullable = true)
 |-- nm_regiao_origem: string (nullable = true)
 |-- nr_singular: string (nullable = true)
 |-- sg_empresa_iata: string (nullable 

In [0]:
df_tipado.write.format("delta").mode("overwrite").saveAsTable("voos_tipado")

### Construção das entidades dimensões:

In [0]:
df = spark.sql("select * from voos_tipado")

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Window

def moda_por_coluna(df, chave, coluna):
    w = Window.partitionBy(chave).orderBy(F.desc("count"))
    
    return (
        df.groupBy(chave, coluna)
          .count()
          .withColumn("rn", F.row_number().over(w))
          .filter("rn = 1")
          .select(chave, coluna)
    )

In [0]:
# dim_aeronave
print("dim_aeronave:")
column_id = "id_equipamento"
colunas = ["ds_modelo", "ds_matricula", "sg_equipamento_icao"]
dim_aeronave = None

for col in colunas:
    moda_df = moda_por_coluna(df, column_id, col)
    dim_aeronave = moda_df if dim_aeronave is None else dim_aeronave.join(moda_df, column_id, "left")

print("IDs repetidos:", dim_aeronave.groupBy(column_id).count().where("count > 1").count())
dim_aeronave.limit(5).display()


dim_aeronave:
IDs repetidos: 0


id_equipamento,ds_modelo,ds_matricula,sg_equipamento_icao
5,ANTONOV AN-124 RUSLAN,null,A124
11,AIRBUS A300 B2/B4/C4/F4/FREIGHTER,null,A30B
12,AIRBUS A310,null,A310
13,AIRBUS A318,null,A318
14,AIRBUS A319,TME,A319


--------------------


In [0]:

# dim_aeroporto
print("dim_aeroporto:")
aeroportos = df.select([
    F.col("id_aerodromo_destino").alias("id_aerodromo"),
    F.col("nm_aerodromo_destino").alias("nm_aerodromo"),
    F.col("nm_continente_destino").alias("nm_continente"),
    F.col("nm_municipio_destino").alias("nm_municipio"),
    F.col("nm_pais_destino").alias("nm_pais"),
    F.col("nm_regiao_destino").alias("nm_regiao"),
    F.col("sg_iata_destino").alias("sg_iata"),
    F.col("sg_icao_destino").alias("sg_icao"),
    F.col("sg_uf_destino").alias("sg_uf")
]).union(df.select([
    F.col("id_aerodromo_origem").alias("id_aerodromo"),
    F.col("nm_aerodromo_origem").alias("nm_aerodromo"),
    F.col("nm_continente_origem").alias("nm_continente"),
    F.col("nm_municipio_origem").alias("nm_municipio"),
    F.col("nm_pais_origem").alias("nm_pais"),
    F.col("nm_regiao_origem").alias("nm_regiao"),
    F.col("sg_iata_origem").alias("sg_iata"),
    F.col("sg_icao_origem").alias("sg_icao"),
    F.col("sg_uf_origem").alias("sg_uf")
]))
column_id = "id_aerodromo"
colunas = [
    "nm_aerodromo",
    "nm_continente",
    "nm_municipio",
    "nm_pais",
    "nm_regiao",
    "sg_iata",
    "sg_icao",
    "sg_uf"
]
dim_aeroporto = None

for col in colunas:
    moda_df = moda_por_coluna(aeroportos, column_id, col)
    dim_aeroporto = moda_df if dim_aeroporto is None else dim_aeroporto.join(moda_df, column_id, "left")

print("IDs repetidos:", dim_aeroporto.groupBy(column_id).count().where("count > 1").count())
dim_aeroporto.limit(5).display()


dim_aeroporto:
IDs repetidos: 0


id_aerodromo,nm_aerodromo,nm_continente,nm_municipio,nm_pais,nm_regiao,sg_iata,sg_icao,sg_uf
1,AB DHABI INTERNATIONAL,ÁSIA,AB DHABI INTERNATIONAL,EMIRADOS ÁRABES UNIDOS,null,AUH,OMAA,null
2,ABBOTSFORD INTERNATIONAL AIRPORT,AMÉRICA DO NORTE,"ABBOTSFORD, BRITISH COLUMBIA",CANADÁ,null,YXX,CYXX,null
3,PORT BOUET AIRPORT (FELIX HOUPHOUET BOIGNY INTERNATIONAL AIRPORT),ÁFRICA,ABIDJAN,COSTA DO MARFIM,null,ABJ,DIAP,null
4,GENERAL JUAN N. ÁLVAREZ INTERNATIONAL AIRPORT,AMÉRICA DO NORTE,ACAPULCO/GEN.JUAN ALVAREZ,MÉXICO,null,ACA,MMAA,null
5,KOTOKA INTERNATIONAL AIRPORT,ÁFRICA,ACCRA,GANA,null,ACC,DGAA,null


In [0]:
# dim_empresa
print("dim_empresa:")
column_id = "id_empresa"
colunas = [
    "sg_empresa_icao",
    "sg_empresa_iata",
    "nm_empresa",
    "nm_pais"
] 
  
dim_empresa = None

for col in colunas:
    moda_df = moda_por_coluna(df, column_id, col)
    dim_empresa = moda_df if dim_empresa is None else dim_empresa.join(moda_df, column_id, "left")

print("IDs repetidos:", dim_empresa.groupBy(column_id).count().where("count > 1").count())
dim_empresa.limit(5).display()


dim_empresa:
IDs repetidos: 0


id_empresa,sg_empresa_icao,sg_empresa_iata,nm_empresa,nm_pais
1000002,AEA,UX,AIR EUROPA LINEAS AEREAS SOCIEDAD ANONIMA,ESPANHA
1000004,MWM,WD,MODERN TRANSPORTE AEREO DE CARGA S.A,BRASIL
1000049,VCV,VO,"CONSORCIO VENEZOLANO DE INDUSTRIAS AERONAUTICAS Y SERVICIOS AÉREOS S.A., CONVIASA",VENEZUELA
1000077,LOT,LO,LOT POLISH AIRLINES,POLÔNIA
1000078,CNT,6C,CONNECT LINHAS AÉREAS S.A. (ANTIGA CONNECT TÁXI AÉREO LTDA.),BRASIL


In [0]:
# dim_linha
print("dim_linha:")
column_id = "id_tipo_linha"
colunas = [
    "cd_tipo_linha", 
    "ds_tipo_linha", 
    "ds_natureza_tipo_linha", 
    "ds_servico_tipo_linha"
] 
  
dim_linha = None

for col in colunas:
    moda_df = moda_por_coluna(df, column_id, col)
    dim_linha = moda_df if dim_linha is None else dim_linha.join(moda_df, column_id, "left")

print("IDs repetidos:", dim_linha.groupBy(column_id).count().where("count > 1").count())
dim_linha.limit(5).display()


dim_linha:
IDs repetidos: 0


id_tipo_linha,cd_tipo_linha,ds_tipo_linha,ds_natureza_tipo_linha,ds_servico_tipo_linha
0,X,NÃO IDENTIFICADA,NÃO IDENTIFICADA,NÃO IDENTIFICADO
1,N,DOMÉSTICA MISTA,DOMÉSTICA,PASSAGEIRO
2,C,DOMÉSTICA CARGUEIRA,DOMÉSTICA,CARGUEIRO
3,I,INTERNACIONAL MISTA,INTERNACIONAL,PASSAGEIRO
4,G,INTERNACIONAL CARGUEIRA,INTERNACIONAL,CARGUEIRO


In [0]:
# dim_data
datas = df.select([
    F.col("dt_partida_real").alias("dt_voo")
]).union(df.select([
    F.col("dt_chegada_real").alias("dt_voo")
]))

column_id = "dt_voo"
datas = datas.distinct()
dim_data = datas\
    .withColumn("dt_voo", F.to_date(F.col("dt_voo")))\
    .withColumn("nr_dia", F.day("dt_voo"))\
    .withColumn("nr_mes", F.month("dt_voo"))\
    .withColumn("nr_ano", F.year("dt_voo"))\
    .withColumn("nr_semestre", F.when(F.col("nr_mes") <= 6, 1).otherwise(2))\
    .withColumn("nr_trimestre", F.ceil(F.col("nr_mes") / 3))

print("IDs repetidos:", dim_data.groupBy(column_id).count().where("count > 1").count())
dim_data.limit(5).display()
    

IDs repetidos: 0


dt_voo,nr_dia,nr_mes,nr_ano,nr_semestre,nr_trimestre
2022-10-28,28,10,2022,2,4
2024-10-26,26,10,2024,2,4
2024-11-04,4,11,2024,2,4
2023-06-12,12,6,2023,1,2
2023-04-19,19,4,2023,1,2


### Construção da tabela fato:

In [0]:
column_id = "id_basica"
fact_voo = df.select([
  "id_basica",
  "nr_voo",
  "nr_singular",
  "ds_natureza_etapa",
  "lt_combustivel", 
  "nr_assentos_ofertados",
  "kg_payload",
  "km_distancia",
  "nr_passag_pagos",
  "nr_passag_gratis", 
  "kg_bagagem_livre",
  "kg_bagagem_excesso",
  "kg_carga_paga",
  "kg_carga_gratis",
  "kg_correio",
  "nr_horas_voadas",
  "kg_peso",
  "nr_velocidade_media",
  "nr_pax_gratis_km",
  "nr_carga_paga_km",
  "nr_carga_gratis_km",
  "nr_correio_km",
  "nr_bagagem_paga_km",
  "nr_bagagem_gratis_km",
  "nr_ask",
  "nr_rpk",
  "nr_atk",
  "dt_partida_real",
  "dt_chegada_real",
  "id_tipo_linha",
  "id_empresa",
  "id_aerodromo_origem",
  "id_aerodromo_destino",
  "id_equipamento",
])


fact_voo = fact_voo.withColumnRenamed("id_aerodromo_origem", "pk_id_aerodromo_partida")
fact_voo = fact_voo.withColumnRenamed("id_aerodromo_destino", "pk_id_aerodromo_chegada")
fact_voo = fact_voo.withColumnRenamed("id_equipamento", "pk_id_equipamento")
fact_voo = fact_voo.withColumnRenamed("id_empresa", "pk_id_empresa")
fact_voo = fact_voo.withColumnRenamed("id_tipo_linha", "pk_id_tipo_linha")
fact_voo = fact_voo.withColumnRenamed("dt_partida_real", "pk_id_dt_partida")
fact_voo = fact_voo.withColumnRenamed("dt_chegada_real", "pk_id_dt_chegada")

print("IDs repetidos:", fact_voo.groupBy(column_id).count().where("count > 1").count())
fact_voo.limit(5).display()


IDs repetidos: 0


id_basica,nr_voo,nr_singular,ds_natureza_etapa,lt_combustivel,nr_assentos_ofertados,kg_payload,km_distancia,nr_passag_pagos,nr_passag_gratis,kg_bagagem_livre,kg_bagagem_excesso,kg_carga_paga,kg_carga_gratis,kg_correio,nr_horas_voadas,kg_peso,nr_velocidade_media,nr_pax_gratis_km,nr_carga_paga_km,nr_carga_gratis_km,nr_correio_km,nr_bagagem_paga_km,nr_bagagem_gratis_km,nr_ask,nr_rpk,nr_atk,pk_id_dt_partida,pk_id_dt_chegada,pk_id_tipo_linha,pk_id_empresa,pk_id_aerodromo_partida,pk_id_aerodromo_chegada,pk_id_equipamento
27557085,1301,999,INTERNACIONAL,0,0,54000,2614,0,0,0,0,6654,0,0,null,6654,null,0,17393600,0,0,0,0,0,0,141156,2023-11-27,2023-11-27,0,1001047,301,673,113
27556890,1505,999,INTERNACIONAL,0,0,54000,1138,0,0,0,0,8104,0,0,null,8104,null,0,9222350,0,0,0,0,0,0,61452,2023-11-18,2023-11-18,0,1001047,136,673,113
27556891,1505,999,INTERNACIONAL,0,0,54000,1717,0,0,0,0,15506,0,0,null,15506,null,0,26623800,0,0,0,0,0,0,92718,2023-11-18,2023-11-18,0,1001047,162,136,113
26764846,800,000,INTERNACIONAL,0,0,108000,9919,0,0,0,0,0,0,0,null,0,null,0,0,0,0,0,0,0,0,1071250,2023-11-19,2023-11-20,0,1006984,301,417,106
26764841,541,000,INTERNACIONAL,0,0,108000,5660,0,0,0,0,94855,0,0,null,94855,null,0,536879000,0,0,0,0,0,0,611280,2023-11-25,2023-11-25,0,1006984,313,16288,106


In [0]:
print("Total de registros na tabela dim_aeronave", dim_aeronave.count())
print("Total de registros na tabela dim_aeroporto", dim_aeroporto.count())
print("Total de registros na tabela dim_empresa", dim_empresa.count())
print("Total de registros na tabela dim_linha", dim_linha.count())
print("Total de registros na tabela dim_data", dim_data.count())
print("Total de registros na tabela fact_voo", fact_voo.count())

Total de registros na tabela dim_aeronave 75
Total de registros na tabela dim_aeroporto 644
Total de registros na tabela dim_empresa 174
Total de registros na tabela dim_linha 5
Total de registros na tabela dim_data 1802
Total de registros na tabela fact_voo 3786075


### Adicionando comentários as colunas:

In [0]:
print("Comentando a tabela dim_aeronave")

tabela = "mvp.silver.dim_aeronave"
descricao_tabela = "A tabela contém dados das aeronaves do dataframe original."
esquema = "mvp"
tabela = "silver.dim_aeronave"
colunas = [
    ("id_equipamento", "Código numérico identificador da aeronave no sistema."),
    ("ds_matricula", "Marca de matrícula da aeronave."),
    ("ds_modelo", "Descrição do modelo da aeronave."),
    ("sg_equipamento_icao", "Designador ICAO do modelo da aeronave (“Type Designator”)"),
]

spark.sql(f"COMMENT ON TABLE {tabela} IS '{descricao_tabela}'")

for col, comment in colunas:
    spark.sql(f"COMMENT ON COLUMN {tabela}.{col} IS '{comment}'")

Cmentando a tabela dim_aeronave


In [0]:
print("Comentando a tabela dim_aeroporto")

tabela = "mvp.silver.dim_aeroporto"
descricao_tabela = "A tabela contém dados dos aeroportos do dataframe original."
esquema = "mvp"
tabela = "silver.dim_aeroporto"
colunas = [
    ("id_aerodromo", "Código identificador do aeródromo no sistema."),
    ("nm_aerodromo", "Nome do aeródromo."),
    ("nm_continente", "Nome do continente em que o aeródromo fica localizado."),
    ("nm_municipio", "Nome do municipio em que o aeródromo fica localizado"),
    ("nm_pais", "Nome do pais em que o aeródromo fica localizado"),
    ("nm_regiao", "Nome da região em que o aeródromo fica localizado"),
    ("sg_iata", "Sigla IATA do aeródromo."),
    ("sg_icao", "Sigla ICAO do aeródromo."),
    ("sg_uf", "Sigla da Unidade Federativa em que o aeródromo fica localizado"),
]

spark.sql(f"COMMENT ON TABLE {tabela} IS '{descricao_tabela}'")

for col, comment in colunas:
    spark.sql(f"COMMENT ON COLUMN {tabela}.{col} IS '{comment}'")

Cmentando a tabela dim_aeroporto


In [0]:
print("Comentando a tabela dim_empresa")

tabela = "mvp.silver.dim_empresa"
descricao_tabela = "A tabela contém dados das empresas responsáveis pelos voos."
esquema = "mvp"
tabela = "silver.dim_empresa"
colunas = [
    ("id_empresa", "Código numérico identificador da empresa no sistema."),
    ("nm_empresa", "Nome da empresa aérea."),
    ("nm_pais", "Nome do país da nacionalidade da empresa aérea."),
    ("sg_empresa_iata", "Sigla IATA da empresa aérea. Refere-se ao designador da empresa de transporte aéreo obtido junto à IATA (Associação Internacional de Transporte Aéreo)."),
    ("sg_empresa_icao", "Sigla ICAO da empresa aérea. Refere-se ao designador da empresa de transporte aéreo obtido junto à OACI (Organização da Aviação Civil Internacional)."),
]

spark.sql(f"COMMENT ON TABLE {tabela} IS '{descricao_tabela}'")

for col, comment in colunas:
    spark.sql(f"COMMENT ON COLUMN {tabela}.{col} IS '{comment}'")

Cmentando a tabela dim_empresa


In [0]:
print("Comentando a tabela dim_linha")

tabela = "mvp.silver.dim_linha"
descricao_tabela = "A tabela contém dados dos tipos de linhas de voo."
esquema = "mvp"
tabela = "silver.dim_linha"
colunas = [
    ("id_tipo_linha", "Código numérico identificador do tipo de linha no sistema."),  
    ("cd_tipo_linha", "Caractere que identifica o tipo de linha. O Tipo de Linha identifica o tipo de operação realizada no voo, podendo ser classificado em: a) 0 (zero) - Etapa Regular; b) 2 (dois) - Etapa Extra; c) 3 (três) -Etapa de Retorno; d) 4 (quatro) -Inclusão de Etapa; e) 6 (seis) -Etapa Não Remunerada Sem Transporte de Objetos; f) 7 (sete) -Etapa de Voo de Fretamento; g) 9(nove) -Etapa de Voo Charter;h) D -Etapa de Voo Duplicada; i) E - Etapa Não Remunerada Com Transporte de Objetos."),
    ("ds_natureza_tipo_linha", "Descrição da natureza referente ao tipo de linha (Internacional/Doméstica). Considera a natureza do voo, não apenas da etapa."),
    ("ds_servico_tipo_linha", "Descrição do serviço referente ao tipo de linha (Passageiro/Cargueiro)."),
    ("ds_tipo_linha", "Descrição do tipo de linha."),
]

spark.sql(f"COMMENT ON TABLE {tabela} IS '{descricao_tabela}'")

for col, comment in colunas:
    spark.sql(f"COMMENT ON COLUMN {tabela}.{col} IS '{comment}'")

Cmentando a tabela dim_linha


In [0]:
print("Comentando a tabela dim_data")

tabela = "mvp.silver.dim_data"
descricao_tabela = "A tabela contém datas dos voos."
esquema = "mvp"
tabela = "silver.dim_data"
colunas = [
    ("dt_voo", "Data de partida ou chegada de um voo."),  
    ("nr_dia", "O dia relativo a data."),
    ("nr_mes", "O mês relativo a data."),
    ("nr_ano", "O ano relativo a data."),
    ("nr_semestre", "O semestre relativo a data."),
    ("nr_trimestre", "O trimestre relativo a data."),
]

spark.sql(f"COMMENT ON TABLE {tabela} IS '{descricao_tabela}'")

for col, comment in colunas:
    spark.sql(f"COMMENT ON COLUMN {tabela}.{col} IS '{comment}'")

Cmentando a tabela dim_data


In [0]:
print("Comentando a tabela fact_voo")

tabela = "mvp.silver.fact_voo"
descricao_tabela = "A tabela contém dados de voos operados em com origem ou destino nacional."
esquema = "mvp"
tabela = "silver.fact_voo"
colunas = [
    ("id_basica", "Código numérico identificador da etapa básica no sistema."),
    ("nr_voo", "Número do voo. Refere-se ao número atribuído à operação de uma etapa ou de uma série de etapas registradas sob a mesma numeração de voo."),
    ("nr_singular", "Singularidade do Voo. Refere-se ao conjunto de caracteres que auxilia na identificação do voo, composto de letras e números escolhidos a critério da própria empresa aérea."),
    ("ds_natureza_etapa", "Descrição da natureza da etapa de voo (Internacional/Doméstica)."),
    ("lt_combustivel", "Quantidade de combustível consumida, expressa em litros."),
    ("nr_assentos_ofertados", "Número de assentos disponíveis na etapa do voo."),
    ("kg_payload", "Capacidade Payload, expressa em quilogramas."),
    ("km_distancia", "Distância, expressa em quilômetros, entre os aeródromos de origem e destino da etapa, considerando a curvatura do planeta terra."),
    ("nr_passag_pagos", "Número de passageiros que ocupam assentos comercializados ao público e que geram receita, com a compra de assentos, para a empresa de transporte aéreo."),
    ("nr_passag_gratis", "Número de passageiros que ocupam assentos comercializados ao público, mas não geram receita, com a compra de assentos, para a empresa de transporte aéreo."),
    ("kg_bagagem_livre", "Bagagem franqueada. É toda bagagem que não é considerada como excesso, expressa em quilogramas."),
    ("kg_bagagem_excesso", "Excesso de bagagem. É a quantidade total de bagagem que excede o limite de peso acordado entre a empresa aérea e o passageiro (adquirido antes ou depois da compra do bilhete), verificada no momento do despacho, expressa em quilogramas."),
    ("kg_carga_paga", "Carga paga. É a quantidade total, expressa em quilogramas, de todos os bens que tenham sido transportados na aeronave, exceto correio e bagagem, e tenham gerado receita direta ou indireta para a empresa aérea."),
    ("kg_carga_gratis", "Quantidade total, expressa em quilogramas, de todos os bens que tenham sido transportados na aeronave, exceto correio e bagagem, e não tenha gerado receitas diretas ou indiretas para a empresa aérea."),
    ("kg_correio", "Quantidade, expressa em quilogramas, de objetos transportados para atender aos operadores designados oficialmente pelo país para operar serviços postais e cumprir com as obrigações associadas decorrentes dos Atos da Universal Postal Union (UPU)."),
    ("nr_horas_voadas", "Número de horas voadas."),
    ("kg_peso", "Refere-se ao peso total carregado pela aeronave, calculado pela soma de carga, correio, passageiros e bagagem, expressos em kg. O peso dos passageiros transportados é calculado multiplicando-se a quantidade total de passageiros por 75, para empresas brasileiras. No caso das empresas estrangeiras, que não enviam dados de bagagem à ANAC, o peso total de passageiros e bagagens é estimado multiplicando-se a quantidade total de passageiros por 90."),
    ("nr_velocidade_media", "Velocidade média do voo, calculada a partir da distância e da duração do voo, expressa em km/h."),
    ("nr_pax_gratis_km", "Representa, para cada etapa básica, o produto entre a quantidade de passageiros grátis e a distância da etapa."),
    ("nr_carga_paga_km", "Refere-se ao produto entre a quantidade de carga paga (kg) e a distância da etapa básica (km)."),
    ("nr_carga_gratis_km", "Refere-se ao produto entre a quantidade de carga grátis (kg) e a distância da etapa básica (km)."),
    ("nr_correio_km", "Refere-se ao produto entre a quantidade de correio (kg) e a distância da etapa básica (km)."),
    ("nr_bagagem_paga_km", "Refere-se ao produto entre a quantidade de bagagem paga (kg) e a distância da etapa básica (km)."),
    ("nr_bagagem_gratis_km", "Refere-se ao produto entre a quantidade de bagagem grátis (kg) e a distância da etapa básica (km)."),
    ("nr_ask", "Refere-se ao volume de Assentos Quilômetros Oferecidos (Available Seat Kilometer). É calculado, em cada etapa básica, pelo produto entre o número de assentos oferecidos e a distância da etapa."),
    ("nr_rpk", "Refere-se ao volume de Passageiros Quilômetros Transportados (Revenue Passenger Kilometer). É calculado, em cada etapa básica, pelo produto entre o número de passageiros pagos e a distância da etapa."),
    ("nr_atk", "Refere-se ao volume de Toneladas Quilômetros Oferecidas (Available Tonne Kilometer), expresso em (ton x km). É calculado, em cada etapa básica, pelo produto entre a Capacidade Payload (kg) e a distância da etapa, dividido por 1000."),
    ("pk_id_dt_partida", "Chave extrangeira que indica qual a data de partida do voo."),
    ("pk_id_dt_chegada", "Chave estrangeira que indica qual a data de chegada do voo."),
    ("pk_id_tipo_linha", "Chave estrangeira que indica o tipo de linha do voo."),
    ("pk_id_empresa", "Chave estrangeira que indica a empresa do voo."),
    ("pk_id_aerodromo_partida", "Chave estrangeira que indica o aeroporto de origem do voo."),
    ("pk_id_aerodromo_chegada", "Chave estrangeira que indica o aeroporto de chegada do voo."),
    ("pk_id_equipamento", "Chave estrangeira que indica a aeronave utilizada no voo."),
]

spark.sql(f"COMMENT ON TABLE {tabela} IS '{descricao_tabela}'")

for col, comment in colunas:
    spark.sql(f"COMMENT ON COLUMN {tabela}.{col} IS '{comment}'")

Comentando a tabela fact_voo


### Persistência das tabelas:

In [0]:
dim_aeronave.write.format("delta").mode("overwrite").saveAsTable("dim_aeronave")
dim_aeroporto.write.format("delta").mode("overwrite").saveAsTable("dim_aeroporto")
dim_empresa.write.format("delta").mode("overwrite").saveAsTable("dim_empresa")
dim_linha.write.format("delta").mode("overwrite").saveAsTable("dim_linha")
dim_data.write.format("delta").mode("overwrite").saveAsTable("dim_data")
fact_voo.write.format("delta").mode("overwrite").saveAsTable("fact_voo")